In [35]:
import pandas as pd
import json
import numpy as np
import os
from pluralizer import Pluralizer
import csv

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [36]:
df = \
( pd.read_json('dwts_scrapy/scores.json')
  .assign(performance_id = lambda df_: np.arange(df_.shape[0]))
  .replace(r'^\s*$', np.nan, regex=True)
  .drop( df.query(" couple== ['Dance-off','Dance Duel'] ").index)
  .rename(columns = {'judge': 'judge_excluded', "dance": "dance_style", "dance_&_theme": "dance_and_theme"})
  .dropna(subset=['couple'])
  .assign(judge = lambda df_: df_.judge_phrase.str.strip(to_strip=" .\n").str.split(", ", expand = False))
  .assign(week = lambda df_: df_.week_title.str.extract(r'Week (\d+)'), #NaN if no Week Theme.
          week_theme = lambda df_: df_.week_title.str.extract(r': (.*?)$').fillna('No Theme')
         )
   # "Caveman" Hustle 
   # Argentine tango
  .assign(michael_jackson_theme = lambda df_: df_.dance_and_theme.str.extract(r'"(.*?)"'),
          michael_jackson_dance = lambda df_: df_.dance_and_theme.str.extract(r'(?:".*?")?(.*?)$'),
          dance_style = lambda df_: np.where(df_.dance_and_theme.notna(), df_.michael_jackson_dance, df_.dance_style)
         )
  .drop(columns = ['michael_jackson_dance'])
  .assign(dance_styles = lambda df_: df_.dance_style.str.split("---"),
         primary_dance_style = lambda df_: df_.dance_styles.apply(lambda x: x[0]))
  .assign(primary_dance_style = lambda df_: df_.primary_dance_style.str.replace("&",'').str.strip().str.title())
 
)


In [37]:
# coalesce 


music_cols = ['elvis_music', 'classical_music', 'motown_music', 'stevie_wonder_music', 
              'music_by_britney_spears', 'music_from_grease', 'cher_music',
              '2022-11-012022-11-01music_by_queen', 'music_by_janet_jackson', 'michael_buble_music']

# some have the music in songs
# df['music'] = df['music'].fillna(df['songs'])


df['music'] = np.where(df['elvis_music'].notna(), df['elvis_music'] + "—Elvis Presley", df['music'])
df['music'] = np.where(df['classical_music'].notna(), df['classical_music'], df['music'])
df['music'] = np.where(df['motown_music'].notna(), df['motown_music'], df['music'])
df['music'] = np.where(df['stevie_wonder_music'].notna(), df['stevie_wonder_music'] + "—Stevie Wonder", df['music'])
df['music'] = np.where(df['music_by_britney_spears'].notna(), df['music_by_britney_spears'] + "—Brittany Spears", df['music'])
df['music'] = np.where(df['music_from_grease'].notna(), df['music_from_grease'], df['music'])
df['music'] = np.where(df['cher_music'].notna(), df['cher_music'] + "—Cher", df['music'])
df['music'] = np.where(df['music_by_queen'].notna(), df['music_by_queen'] + "—Queen", df['music'])
df['music'] = np.where(df['music_by_janet_jackson'].notna(), df['music_by_janet_jackson'] + "—Janet Jackson", df['music'])
df['music'] = np.where(df['michael_buble_music'].notna(), df['michael_buble_music'] + "—Michael Buble", df['music'])

df = \
(
    df
    .drop(columns = music_cols)
    .assign(songs = lambda df_: df_.music.str.split("---"),
            primary_song_phrase = lambda df_: df_.songs.apply(lambda x: x[0]))
)

df[['primary_song','primary_song_artist']] = df.primary_song_phrase.str.extract(r'(.*)[—―一]+(.*?)$')


In [17]:
# Some weeks have a technical score/performance score. but no scores column.

# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_11)#Weekly_scores Week 4
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_10)#Weekly_scores Week 4

# double_score = df[df['technical_score'].notna()]

# double_score[double_score.columns[~double_score.isnull().any()]]

In [19]:
# season 22, week 8 has a very difficult judges score arrangement
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_22)#Week_8:_Judges'_Team-up_Challenge

#  total (9, X, 9, 9) where the final score is "america's score" and the X is the score that would have been the score from the judge
# in df['judge'] column.  gah.  but those are multi couple dances.  phew.

In [20]:
# editing on the pages made this moot.
# Some couple columns have additional info which is in parens in the column. This creates an NA couple column.

# pattern = r'(?P<couple_alone>.*?)\((?P<additional>.*?)\)$'

# # e.g., https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_27) Week 4.  Couple (Trio Dance Partner)
# df[['couple_alone','trio_partner']] = df['couple_trio_dance_partner_'].str.extract(pattern)
# df['couple'] = df['couple'].fillna(df['couple_alone'])
# df.drop(columns='couple_alone', inplace=True)

# # https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_16) Week 5 (Professionals).
# df[['couple_alone','additional_dancers']] = df['couple_professionals_'].str.extract(pattern)
# df['couple'] = df['couple'].fillna(df['couple_alone'])
# df.drop(columns='couple_alone', inplace=True)

# # couple_team_up_judge_ from https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_22)#Week_8:_Judges'_Team-up_Challenge
# # Some of These are multi-couple dances, with a judge listed i the couple column, and then an X for the score that the judge that coached the team would have given.
# # but others are regular dances (just a different column name)
# df['couple'] = df['couple'].fillna(df['couple_team_up_judge_'])

# # couple_judge_  from https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_11)#Week_10:_Finals is much simpler (just a judge chosen dance).
# df[['couple_alone','judge_choosing_dance']] = df['couple_judge_'].str.extract(pattern)
# df['couple'] = df['couple'].fillna(df['couple_alone'])
# df.drop(columns='couple_alone', inplace=True)

# # One remains from the couple_judge_ due to a substitution which then didn't have a judge (so the regex above doesn't catch it)
# # 2341 where the couple_judge_ is Nastia & Sasha[a]
# df['couple'] = df['couple'].fillna(df['couple_judge_'])
# # df.iloc[2341]

# df.drop(columns=['couple_trio_dance_partner_', 'couple_professionals_', 'couple_judge_'], inplace=True)

# df[df['couple'].isna()]

In [38]:
# Some have technical_score and performance_score.
# add a score_type column.
# split these out, going to be two rows, each with scores column and a score_type column.

double_scores = ( df[df['technical_scores'].notna()]
                  .drop(columns="scores")
                  .dropna(axis = 1, how = "all") )

val_cols = ['technical_scores','performance_scores']
id_cols = [ele for ele in double_scores if ele not in val_cols]
# now get two rows per
double_scores = double_scores.melt(id_vars = id_cols, value_vars = val_cols, value_name = "scores", var_name = "score_type")

single_scores = df.drop(df.index[df['technical_scores'].notna()])
# make others be score_type
single_scores['score_type'] = "single_score"
single_scores.drop(columns=['technical_scores','performance_scores'], inplace=True)

df_score_type = pd.concat([double_scores, single_scores])


In [39]:
# df_score_type.head()

In [40]:
df_score_type[['total_score','indiv_scores']] = df_score_type['scores'].str.extract(r'(?P<total_score>\d+)\s\((?P<indiv_scores>.*)\)')

# new_cols['judge_score_list'] = new_cols['indiv_scores'].str.split(",")


# df_new = pd.concat([df, new_cols], axis = 1)

In [41]:
# song_cols = df['music'].str.extract(r'"(?P<song_name>.*)"—(?P<song_artist>.*)$')
# song_cols
# df["music"].str.split(" / ") Some music columns have multiples.

In [42]:
# most are multi couple dances, with ranking scores.

df_score_type["couple_list"] = df_score_type["couple"].str.split("---")
df_score_type["couple_count"] = df_score_type["couple_list"].apply(len)



In [43]:
# Now eliminate multi-couple dances and dances with NA total scores (mostly dance offs and others). 
#  Might still be one or two issues with total scores but not individual scores.

df_single_couple = df_score_type.loc[(df_score_type['couple_count'] == 1) & df_score_type['total_score'].notna() ].copy()

# df_single_couple.head()



In [44]:
# df_single_couple[df_single_couple['indiv_scores'].isna()]

In [45]:
# df_single_couple.head()

df_single_couple['judge_score_list'] = df_single_couple['indiv_scores'].str.split(", ")

# df_single_couple['judge_score_list']
df_single_couple['judge_score_count'] = df_single_couple['judge_score_list'].apply(len)

df_single_couple['judge_count'] = df_single_couple['judge'].apply(len)


# a few have a total score, but not an individual score.  e.g., s9w10.

In [29]:
#pd.set_option('max_colwidth', None) 
#df_single_couple[df_single_couple['season'] == 31][['week','judge_phrase']]

In [30]:
# df_single_couple.loc[df_single_couple['judge_score_count'] != df_single_couple['judge_count']]

# Trouble in 
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_23)#Week_5:_Most_Memorable_Year_Night
# https://en.wikipedia.org/wiki/Dancing_with_the_Stars_(American_season_31)#Week_6:_Michael_Bubl%C3%A9_Night

In [46]:
df_final = ( df_single_couple
              #  .head(5)
               .explode(['judge','judge_score_list'])
               .rename(columns={'judge_score_list': 'judge_score'})
               .drop(columns=['indiv_scores','judge_phrase','scores'])
               .assign(judge = lambda df: df.judge.str.strip(" ."))
)

df_final.drop( df_final.query(" judge_score == 'X' ").index, inplace = True)

df_final['judge_score_id'] = np.arange(df_final.shape[0])

In [47]:
# rename "Guest judge" with value from df_final['guest_judge']
df_final['judge'] = np.where(df_final['judge'] == 'Guest judge', df_final['guest_judge'], df_final['judge'])

# fix Julianne Hough (Night 1 only)
df_final['judge'] = np.where(df_final['judge'] == 'Julianne Hough (Night 1 only)', 'Julianne Hough', df_final['judge'])

# sorted(df_final['judge'].unique().tolist())

In [48]:
df_final['judge_score'] = df_final['judge_score'].astype(float)

In [49]:
sorted(df_final.columns)

['broadway_show',
 'celeb_name_part',
 'celebrity',
 'cirque_du_soleil_show',
 'couple',
 'couple_count',
 'couple_list',
 'dance_and_theme',
 'dance_chosen_by',
 'dance_style',
 'dance_styles',
 'disney_film',
 'dynamic_duo',
 'era',
 'film',
 'film_series',
 'film_theme',
 'genre',
 'guest_judge',
 'horror_film_show',
 'icon',
 'icons',
 'james_bond_film',
 'judge',
 'judge_count',
 'judge_excluded',
 'judge_score',
 'judge_score_count',
 'judge_score_id',
 'mentor',
 'michael_jackson_theme',
 'music',
 'musical',
 'notability',
 'original_couple',
 'performance_id',
 'primary_dance_style',
 'primary_song',
 'primary_song_artist',
 'primary_song_phrase',
 'pro_name_part',
 'professional',
 'professionals',
 'result',
 'score_type',
 'season',
 'songs',
 'television_show',
 'total_score',
 'trio_dance_partner',
 'trio_partner',
 'tv_show',
 'villain',
 'week',
 'week_theme',
 'week_title']

In [50]:
df_agg = df_final.groupby(["performance_id", "judge"]).agg({'judge_score':"mean"}).reset_index()

In [51]:
df_agg.head()

,performance_id,judge,judge_score
0,0,Bruno Tonioli,6.0
1,0,Carrie Ann Inaba,7.0
2,0,Len Goodman,5.0
3,1,Bruno Tonioli,7.0
4,1,Carrie Ann Inaba,5.0


In [52]:
# just want one row for each group, so we can join back to it.
df_summarized = ( df_final.drop(columns=['score_type','judge_score'])
                        .groupby(["performance_id", "judge"]).head(1)
                )
# 

In [53]:
df_summarized2 = df_summarized.merge(df_agg, how = 'left', on = ["performance_id", "judge"])

In [54]:
df_final = df_summarized2.copy()



In [40]:
# Now need to group_by and average scores to get rid of the technical_scores and performance_scores.

In [55]:
all_cols = df_final.columns.values.tolist()

to_remove = ['judge_excluded', 'couple_list', 'couple_count', 'judge_score_count', 
             'judge_count', 'dance_styles', 'songs', 'primary_song_phrase', 'dance_&_theme']

all_cols = [ele for ele in all_cols if ele not in to_remove]

front_cols = [ 'judge_score_id', 'judge','judge_score', 'performance_id','season','week','week_theme', 'couple',
              'primary_dance_style','primary_song', 'primary_song_artist', 'total_score' ]

remaining_cols = [ele for ele in all_cols if ele not in front_cols]

reordered = front_cols + remaining_cols

In [56]:
df_final = df_final[reordered]

In [43]:
df_final.to_csv("dwts_dataset.csv", index = False)

In [57]:
df_final.shape

(9733, 47)

# Produce tables

```
performances (id, song_id, dance_style_id, couple_id, season_id, week_id, week_theme_id)
dance_styles (id, name) 
songs (id, name, artist_id)
artists (id, name)
scores (id, performance_id, person_id, score)
seasons (id, season_no, year_broadcast)
week_themes (id, name, season_id)
              
couples (id, name, celebrity_person_id, pro_person_id) # maybe.

people # Note that some work needed to match up couples to people.
```

In [46]:
# dance_styles_series = df_final['dance_style'].unique()
# dance_styles = pd.DataFrame({'id': np.arange(1, len(dance_styles_series) + 1), 'name': dance_styles_series})
# dance_styles.to_csv("dwts_dataset/dance_styles.csv", index = False)

In [47]:
# couples_series = df_final['couple'].unique()
# couples = pd.DataFrame({'id': np.arange(1, len(couples_series) + 1), 'name': couples_series})
# couples.to_csv("dwts_dataset/couples.csv", index = False)

In [59]:
pluralizer = Pluralizer()

for dim in ['couple','primary_dance_style', 'week_theme', 'judge', 'professional', 'primary_song_artist']:
    df_final[dim + "_id"] = df_final[dim].factorize()[0] + 1
    filename = "dwts_dataset/" + pluralizer.plural(dim) + ".csv"
    # os.remove(filename)
    df_final[[dim + "_id", dim]].drop_duplicates().to_csv(filename, index = False, quoting=csv.QUOTE_NONNUMERIC)


# celebrity needs known_for
df_final['celebrity_id'] = df_final['celebrity'].factorize()[0] + 1
filename = "dwts_dataset/" + pluralizer.plural('celebrity') + ".csv"
df_final[['celebrity_id', "celebrity", 'notability']].drop_duplicates(subset=['celebrity_id', 'celebrity']).to_csv(filename, index = False, quoting=csv.QUOTE_NONNUMERIC)
    
# song needs primary_artist
df_final['primary_song_id'] = df_final['primary_song'].factorize()[0] + 1
filename = "dwts_dataset/" + pluralizer.plural('primary_song') + ".csv"
df_final[['primary_song_id', "primary_song", 'primary_song_artist_id']].drop_duplicates(subset=['primary_song_id', "primary_song"]).to_csv(filename, index = False, quoting=csv.QUOTE_NONNUMERIC)
# 
# df_final[['couple_id','couple']].drop_duplicates().to_csv("dwts_dataset/couples.csv", index = False)2022-10-312022-10-31

In [60]:
# need to have 'primary_song' and 'primary_song_artist' such that 'primary_song_artist_id' is in primary_songs
# df_final[['celebrity_id', "celebrity", 'notability']].drop_duplicates(subset=['celebrity_id', 'celebrity'])

In [61]:
performances = df_final[['performance_id', 'primary_song_id', 'primary_dance_style_id', 
                         'couple_id', 'professional_id', 'celebrity_id','season', 'week', 'week_theme_id']]
performances.drop_duplicates().to_csv("dwts_dataset/performances.csv", index = False, quoting=csv.QUOTE_NONNUMERIC)

scores = df_final[['performance_id', 'judge_id', 'judge_score']].copy()
# scores.drop_duplicates(inplace = True)

scores['score_id'] = np.arange(1, len(scores) + 1)
scores = scores[['score_id', 'performance_id', 'judge_id', 'judge_score']]
scores.to_csv("dwts_dataset/scores.csv", index = False, quoting=csv.QUOTE_NONNUMERIC)